In [ ]:
import pandas as pd
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import gc
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
import warnings
warnings.filterwarnings('ignore')
train =True

### この実験で、investment_id毎に学習させると、スコアは悪くなることがわかりました。
### 別のアプローチが必要のようです。

### この実験で、投資_id毎に学習させる、スコアは悪くなることがわかりました。
### 別のアプローチに必要のあるようです。

In [ ]:
%%time
data_types_dict = {
    'time_id': 'int32',
    'investment_id': 'int16',
    "target": 'float16',
}

features = [f'f_{i}' for i in range(300)]

for f in features:
    data_types_dict[f] = 'float16'
    
target = 'target'
train = pd.read_csv('/kaggle/input/ubiquant-market-prediction/train.csv', 
                       usecols = data_types_dict.keys(),
                       dtype=data_types_dict, 
                       index_col = 0)
train

In [ ]:
len(sorted(train.investment_id.unique()))

In [ ]:
max(train.investment_id.unique())

### investment_idのデータ量が多い上位500個だけ抽出。処理の高速化のためです。

### Extract only the top 500 investment_id's with the largest amount of data.This is to speed up the process

In [ ]:
investment = train.investment_id.value_counts()
investment = investment.index[:500]

In [ ]:
investment

In [ ]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score
#!pip install optuna
import optuna 
import optuna.integration.lightgbm as lgbo

In [ ]:
import pickle
with open('../input/ubiquant-model/models', 'rb') as web:
    models = pickle.load(web)

In [ ]:
import pickle
with open('../input/ubiquant-model/reg', 'rb') as web:
    reg = pickle.load(web)

### trainデータにないinvestment_idが出現するようです。

### It seems that an investment_id that is not in the train data appears.

In [ ]:
%time
import ubiquant
env = ubiquant.make_env()  
iter_test = env.iter_test()
for (test_df, sample_prediction_df) in iter_test: 
        
  for i in investment:
    test_x =test_df[test_df.investment_id == i]
    if len(test_x) !=0:
    
      test_x = test_x.reset_index(drop=True)
      row_id = test_x.row_id
      test_x = test_x.drop(['row_id'],axis=1)
    
      test_x['pred'] = models[i].predict(test_x)
    
      test_x['row_id'] = row_id
    
      for j,row in test_x.iterrows():
            sample_prediction_df.loc[sample_prediction_df['row_id'] == row['row_id'], 'target'] = row.pred
   
  #Other
  other =[]
  for i in  test_df.investment_id.unique():
    if i in  investment.values:
        continue
    else:
        other.append(i)
  
  if len(other)!=0:
      test_x = pd.DataFrame()
      for i in other:
          test_x = pd.concat([test_x,test_df[test_df.investment_id == i]])
    
      test_x = test_x.reset_index(drop=True)
      row_id = test_x.row_id
      test_x = test_x.drop(['row_id'],axis=1)
     
      test_x['pred'] = reg.predict(test_x)
    
      test_x['row_id'] = row_id
    
      for j,row in test_x.iterrows():
            sample_prediction_df.loc[sample_prediction_df['row_id'] == row['row_id'], 'target'] = row.pred
        
  env.predict(sample_prediction_df) 
  #sample_prediction_df

In [ ]:
 sample_prediction_df